In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
from sklearn import preprocessing
import numpy as np
import lightgbm as lgb
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import cohen_kappa_score
from functools import partial
import scipy as sp

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Los objetivos de este analisis son :**


1. Conocer las variables mas importantes para predecir la readmision en pacientes diabeticos y que el modelo hipotetico sea de alta prediccion, del menor numero de variables posible para que pueda ser explicado y totalmente implementables.

2. Detallar como son los experimentos con que se llego a los resultados

3. Explorar algunas tecnicas de feature engineering segun sea el tipo de datos que se encontrara

4. Explorar como la seleccion de variables ayuda a mejorar la prediccion y a la vez ayudar a la explicacion del modelo al hacerlo mas compacto

In [2]:
df_data=pd.read_csv('../input/diabetes-globant/diabetic_data.csv')

In [3]:
df_data.columns

Confirmemos si la id 'encounter_id' es unica y si el id del paciente se repite.

In [4]:
print (len(df_data['encounter_id'].unique())/len(df_data))

print (len(df_data['patient_nbr'].unique())/len(df_data))

In [5]:
df_data['patient_nbr'].value_counts()

Las variables 'age','max_glu_serum','readmitted'(variable a predecir) y 'weight' se convierten a enteros para explotar el orden que tienen sus valores. Luego algunas etiquetas que aparecne en la data se convierten a nulo para unfirmoizar los valores perdidos en la data.

In [6]:
##limpieza

print(df_data['age'].value_counts())

df_data['age']=df_data['age'].map({'[0-10)':1,'[10-20)':2,'[20-30)':3,'[30-40)':4,'[40-50)':5,'[50-60)':6,'[60-70)':7,'[70-80)':8})

print(df_data['age'].value_counts())

df_data=df_data.replace("?",np.nan)
df_data=df_data.replace("None",np.nan)

df_data.loc[df_data['max_glu_serum']=='Norm','max_glu_serum']=1
df_data.loc[df_data['max_glu_serum']=='>200','max_glu_serum']=2
df_data.loc[df_data['max_glu_serum']=='>300','max_glu_serum']=3


df_data.loc[df_data['readmitted']=='NO','readmitted']=0
df_data.loc[df_data['readmitted']=='<30','readmitted']=1
df_data.loc[df_data['readmitted']=='>30','readmitted']=2

 
df_data['weight']=df_data['weight'].map({'[0-25)':1,'[25-50)':2,'[50-75)':3,'[75-100)':4,'[100-125)':5,'[125-150)':6,'[150-175)':7,'[175-200)':8,'>200/':9 })





In [7]:
df_data['readmitted'].value_counts()

Observemos los valores nulos por cada variable:

In [8]:
df_data.isnull().mean()

In [9]:


def entrena_lgb(data,features,categorical,target):

    kfold=KFold(n_splits=5,shuffle=True,random_state=2021)


    i=1

    r=[]

    importancias=pd.DataFrame()

    importancias['variable']=features
    
    
    cat_ind=[features.index(x) for x in categorical if x in features]
    
    dict_cat={}
    
    categorical_numerical = data[categorical].dropna().select_dtypes(include=np.number).columns.tolist()
    
    categorical_transform=[x for x in categorical if x not in categorical_numerical]
    
    for l in categorical_transform:
        le = preprocessing.LabelEncoder()
        le.fit(list(data[l].dropna()))

        dict_cat[l]=le

        data.loc[~data[l].isnull(),l]=le.transform(data.loc[~data[l].isnull(),l])
        
        

    for train_index,test_index in kfold.split(data):

        lgb_data_train = lgb.Dataset(data.loc[train_index,features].values,data.loc[train_index,target].values)
        lgb_data_eval = lgb.Dataset(data.loc[test_index,features].values,data.loc[test_index,target].values, reference=lgb_data_train)

        params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': { 'rmse'},
            "max_depth":-1,
            "num_leaves":32,
            'learning_rate': 0.1,
        "min_child_samples": 100,
            'feature_fraction': 0.9,
         "bagging_freq":1,
            'bagging_fraction': 0.9,
            "lambda_l1":10,
            "lambda_l2":10,
           # "scale_pos_weight":30,

            'verbose': 1    
        }




        modelo = lgb.train(params,lgb_data_train,num_boost_round=13100,valid_sets=lgb_data_eval,early_stopping_rounds=50,verbose_eval=25,categorical_feature=cat_ind)

        importancias['gain_'+str(i)]=modelo.feature_importance(importance_type="gain")


        data.loc[test_index,'estimator']=modelo.predict(data.loc[test_index,features].values, num_iteration=modelo.best_iteration)

        print ("Fold_"+str(i))
        a= (mean_squared_error(data.loc[test_index,target],data.loc[test_index,'estimator']))**0.5
        r.append(a)
        print (a)
        print ("")

        i=i+1
        
    for l in categorical_transform:

            data.loc[~data[l].isnull(),l]=dict_cat[l].inverse_transform(data.loc[~data[l].isnull(),l].astype(int))
            
    importancias["gain_avg"]=importancias[["gain_1","gain_2","gain_3","gain_4","gain_5"]].mean(axis=1)
    importancias=importancias.sort_values("gain_avg",ascending=False).reset_index(drop=True)

    print ("mean: "+str(np.mean(np.array(r))))
    print ("std: "+str(np.std(np.array(r))))
    
    return importancias

In [10]:
print(df_data['max_glu_serum'].value_counts())
print(df_data['diag_1'].value_counts())
print(df_data['diag_2'].value_counts())
print(df_data['diag_3'].value_counts())


 Notar la alta cardinalidad de algunas variables categoricas:

In [11]:
### notar la alta cardinalidad de algunas variables categoricas

for x in ['race','gender','admission_type_id', 'discharge_disposition_id', 'admission_source_id',
             'payer_code', 'medical_specialty','A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone','metformin-pioglitazone','change', 'diabetesMed','diag_1','diag_2','diag_3']:
    
    print(x)
    print(df_data[x].nunique())
    print('')
    
    

# **Primer Experimento - Baseline**

Creare a conintuacion el modelo baseline que sus variables seran tal y como vienen en la data luego de la limpiaza inicial que se realizo.

Para predecir la readmision se opta por una regresion para aprovechar el orden que hay entre los valores, el cual no ocurriria si se optaria por una clasificacion multinomial.

Se realizara una validacion cruzada al no observar un patron de dependencia del tiempo entre las observaciones, luego el modelo se optimizara con rmse y finalmente con un optimizador se convertira los valores decimales que arroja la prediccion a 0 , 1 y 2.

In [12]:


categorical=['patient_nbr','race','gender','admission_type_id', 'discharge_disposition_id', 'admission_source_id',
             'payer_code', 'medical_specialty','A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone','metformin-pioglitazone','change', 'diabetesMed','diag_1','diag_2','diag_3']

no_usar=['encounter_id','readmitted','estimator']

features=[x for x in df_data.columns if x not in no_usar]

importancias=entrena_lgb(data=df_data,features=features,categorical=categorical,target='readmitted')



In [13]:
importancias

Se usa el siguiente optimizador para elegir los puntos de corte que convertiran los valores predicho en 0 , 1 2 en el caso que este modelo tenga que ser pasado a un ambiente productivo. Esto es mucho mas ventajoso que redondear las predicciones solamente al entero mas cercano.

El codigo de dicho optimizador esta basado en el codigo del siguiente enlace:

https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved

In [14]:
## basado en esto https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
class OptRounder(object):
    def __init__(self):
        self.res_ = []
        self.coef_ = []
        
    def get_res(self):
        return self.res_
    
    # objective function
    def func(self, coef, X, y):

        mse = mean_squared_error(self.bincut(coef, X), y)
        return mse
    
    
    def bincut(self, coef, X):
        return pd.cut(X,
                      [-np.inf] + list(np.sort(coef)) + [np.inf],
                      labels = [0, 1, 2])
        
    def fit(self, X, y):
        pfunc = partial(self.func, X=X, y=y)
        self.res_ = sp.optimize.minimize(fun = pfunc,           # objective func
                                         x0 = [0.5, 1.2],  # initial coef
                                         method='nelder-mead')  # solver
        
        
        self.coef_ = self.res_.x
        
    def predict(self, X, coef):
        return self.bincut(coef, X)

In [15]:
optR = OptRounder()
optR.fit(df_data["estimator"].values.reshape(-1,), df_data['readmitted'].astype(int))
res = optR.get_res() 

In [16]:
coefficients = res.x        


(mean_squared_error(optR.predict(df_data["estimator"].values, coefficients).astype(int),df_data['readmitted'].astype(int)))**0.5

Vemos que 0.913 es el valor de nuestra baseline

In [17]:
res

# **Segundo Experimento**

Observamos que en la importancia de variables del primer experimento que 'diag_1','diag_2' y 'diag_3' son las variables mas importantes y tienen alta cardinalidad. En estos casos muchas veces estas variables tienden a overfitear por mas que salgan como las variables mas relevantes. Probemos extrayendo estas variables en nuestro segundo experimento.

In [18]:
no_usar=['encounter_id','readmitted','estimator', 'diag_1','diag_2','diag_3']

features=[x for x in df_data.columns if x not in no_usar]

importancias_2=entrena_lgb(data=df_data,features=features,categorical=categorical,target='readmitted')

Tan solo sacando esas 3 variables el score bajo de 0.870 a 0.866 !

# **Tercer Experimento**

Probamos sacando el promedio y varianza del 'number_patient'(variable numerica mas relevante) en variables de alta cardinalidad , volveremos a poner las 3 variables extraidas anteriormente y veremos mas abajo que el score sera practicamente el mismo de la baseline. El codigo que esta comentado tambien fueron variables que se crearon con la logica hablada pero los resultados seguian siendo basicamente los mismos.

In [19]:
### probamos haciendo algo de feature engineering a las variables mas importantes, en caso alguna salga como relevante, se puede discutir la razon de su relevancia

#for x in [ 'diag_1',
# 'diag_2',
# 'diag_3']:
    
#    df_data[x+"_size"]=df_data.groupby(x)['encounter_id'].transform('size')
    
#for x in [ 'diag_1',
# 'diag_2',
# 'diag_3']:
    
#    df_data[x+"_number_inpatient_mean"]=df_data.groupby(x)['number_inpatient'].transform('mean')
#    df_data[x+"_number_inpatient_var"]=df_data.groupby(x)['number_inpatient'].transform('var')

for x in [ 'discharge_disposition_id',
 'admission_source_id',
 'medical_specialty']:
    
    df_data[x+"_number_inpatient_mean"]=df_data.groupby(x)['number_inpatient'].transform('mean')
    df_data[x+"_number_inpatient_var"]=df_data.groupby(x)['number_inpatient'].transform('var')
      

In [20]:
no_usar=['encounter_id','readmitted','estimator']

features=[x for x in df_data.columns if x not in no_usar]

importancias_2=entrena_lgb(data=df_data,features=features,categorical=categorical,target='readmitted')

Aqui vemos que el score el mismo o incluso ligeramente peor que en la baseline

In [21]:
importancias_2

# **Cuarto Experimento**

Usaremos el **target encoding** para las 6 variables con mas alta cardinalidad pero usando las mismas particiones de la validacion cruzada usada para evitar el **data leakage.**

Recordemos que a diferencia del segundo experimento, no se ha sacado ninguna variable categorica.

La funcion de target encoding creada,hace que cuando el tama;o de muestra es menor a 20, se asigna como valor nulo a la variable creada para evitar efectos de tamaño de muestra pequeño. Dicho tamaño de muestra es una parametro de la funcion que se puede variar.

In [22]:
def target_encoding(data,variable,target,threshold=20,function='mean'):

    list_part=[]


    kfold=KFold(n_splits=5,shuffle=True,random_state=2021)

    for train_index,test_index in kfold.split(data):

        temp=data.loc[train_index].groupby(variable)[target].agg([function,'size']).reset_index()

        temp=temp.loc[temp['size']>threshold].reset_index(drop=True)

        del temp['size'] ; gc.collect()

        temp=temp.rename(columns={function:function+'_enconding_'+variable+"-"+target})
        
        part=data.loc[test_index,['encounter_id',variable]]

        part=part.merge(temp,on=variable,how='left')

        list_part.append(part)

    df_part=pd.concat(list_part,ignore_index=True)
    
    del df_part[variable] ; gc.collect()
    
    data=data.merge(df_part,on='encounter_id',how='left')
    
    return data




    

In [23]:
df_data=target_encoding(data=df_data,variable='diag_1',target='readmitted')
df_data=target_encoding(data=df_data,variable='diag_2',target='readmitted')
df_data=target_encoding(data=df_data,variable='diag_3',target='readmitted')
df_data=target_encoding(data=df_data,variable='discharge_disposition_id',target='readmitted')
df_data=target_encoding(data=df_data,variable='admission_source_id',target='readmitted')
df_data=target_encoding(data=df_data,variable='medical_specialty',target='readmitted')

df_data=target_encoding(data=df_data,variable='diag_1',target='readmitted',function='var')
df_data=target_encoding(data=df_data,variable='diag_2',target='readmitted',function='var')
df_data=target_encoding(data=df_data,variable='diag_3',target='readmitted',function='var')
df_data=target_encoding(data=df_data,variable='discharge_disposition_id',target='readmitted',function='var')
df_data=target_encoding(data=df_data,variable='admission_source_id',target='readmitted',function='var')
df_data=target_encoding(data=df_data,variable='medical_specialty',target='readmitted',function='var')

In [24]:
no_usar=['encounter_id','readmitted','estimator']

features=[x for x in df_data.columns if x not in no_usar]

importancias_3=entrena_lgb(data=df_data,features=features,categorical=categorical,target='readmitted')

A pesar de las variables agregadas, el score continua siendo practicamente el mismo de la baseline.

In [25]:
importancias_3

# **Quinto Experimento**

Si bien vimos que las 3 variables categoricas que sacamos en el segundo experimento estaban overfiteando y tambien puede ser confirmado al regresarlas en los 2 experimentos anteriores. Tal vez algunos de los niveles de dichas variables si podrian tener poder predictivo y podrian estar representados en el target encoding.

Entonces mantengamos las variables creadas  el target encoding pero volvamos a extraer las 3 variables con mas alta cardinalidad como en el segundo experimento y veamos los resultados.

In [26]:
no_usar=['encounter_id','readmitted','estimator','diag_1','diag_2','diag_3']

features=[x for x in df_data.columns if x not in no_usar]

importancias_4=entrena_lgb(data=df_data,features=features,categorical=categorical,target='readmitted')

Observamos que la hipotesis se hace cierta y superamos al score del segundo experimento

In [27]:
importancias_4

Entonces probemos el score optimizando los puntos de corte como se hablo anteriormente

In [28]:
optR = OptRounder()
optR.fit(df_data["estimator"].values.reshape(-1,), df_data['readmitted'].astype(int))
res = optR.get_res() 


coefficients = res.x        

(mean_squared_error(optR.predict(df_data["estimator"].values, coefficients).astype(int),df_data['readmitted'].astype(int)))**0.5



Aqui se confirma que se vuelve a superar el score.

# **Sexto Experimento**

Extraigamos las variables con 'gain_avg' mayor a 0 y esperemos encontrar un score mejor o similar. Lo comprobaremos en los resultados de mas abajo.

In [29]:
features_selected=importancias_4.loc[importancias_4['gain_avg']>0,'variable'].tolist()

print(len(features_selected))

importancias_5=entrena_lgb(data=df_data,features=features_selected,categorical=categorical,target='readmitted')

Observamos que pasando de las 65 variables originales a 50 variables, obtenemos un modelo de resultados muy similares al del mejor escore que es el del experimento anterior.

# **Septimo Experimento**

Seleccionemos variables usando el metodo de target perumutation que basicamente hace que para cada una de las variables es permutar el target muchas veces y compara si el score sin el target permutado tiene diferencia significativa con los score con target permutado. En caso eso sea cierto, la variable queda seleccionada.

La metodologia y implementacion se base en los siguientes enlaces:

https://www.kaggle.com/ogrellier/feature-selection-with-null-importances

https://academic.oup.com/bioinformatics/article/26/10/1340/193348



In [30]:
def get_feature_importance( data,features,categorical,target,shuffle=True, seed=None):

    
    cat_ind=[features.index(x) for x in categorical if x in features]
    
    dict_cat={}
    
    categorical_numerical = data[categorical].dropna().select_dtypes(include=np.number).columns.tolist()
    
    categorical_transform=[x for x in categorical if x not in categorical_numerical]
    
    for l in categorical_transform:
        le = preprocessing.LabelEncoder()
        le.fit(list(data[l].dropna()))

        dict_cat[l]=le

        data.loc[~data[l].isnull(),l]=le.transform(data.loc[~data[l].isnull(),l])
    
    y = data[target].copy()
    if shuffle:
        y = data[target].copy().sample(frac=1.0,random_state=seed)
    
    dtrain = lgb.Dataset(data[features].values, y.values, free_raw_data=False, silent=True)
    lgb_params = {
        'objective': 'regression',
        'boosting_type': 'gbdt',
     "learning_rate": 0.2,
        'metric': { 'rmse'},
        'subsample': 0.9,
        'colsample_bytree': 0.9,
            "min_child_samples": 100,
        'num_leaves': 64,
        'max_depth': -1,
        'seed': seed,
        'bagging_freq': 1,
    }
    
    # Fit the model
    clf = lgb.train(params=lgb_params, train_set=dtrain, num_boost_round=90,categorical_feature=cat_ind)

    # Get feature importances
    imp_df = pd.DataFrame()
    imp_df["feature"] = list(features)
    imp_df["importance_gain"] = clf.feature_importance(importance_type='gain')
    imp_df['trn_score'] = mean_squared_error(y, clf.predict(data[features].values))
    
    for l in categorical_transform:

            data.loc[~data[l].isnull(),l]=dict_cat[l].inverse_transform(data.loc[~data[l].isnull(),l].astype(int))
    
    return imp_df


def get_importance_permutation(data,variables,categorical,target,n_iterations=50):

    for x in range(1,n_iterations):
        temp=get_feature_importance(data=data,features=variables,categorical=categorical,target=target,shuffle=True, seed=x)
        temp["run"]=x

        if x==1:
            general=temp.copy()

        else :
            general=general.append(temp,ignore_index=True)

        print(x)


    ranking=get_feature_importance(data=data,features=variables,categorical=categorical,target=target,shuffle=False, seed=2021)

    features_quantile=general.groupby("feature")["importance_gain"].quantile(0.95).reset_index()

    features_quantile=features_quantile.rename(columns={"importance_gain":"quantile_0.95"})
    ranking=ranking.merge(features_quantile,on="feature",how="left")

    ranking=ranking.sort_values("importance_gain",ascending=False).reset_index(drop=True)
    
    return ranking

In [31]:
importances_6=get_importance_permutation(data=df_data,variables=features_selected,categorical=categorical,target='readmitted')

In [36]:
importances_6

In [39]:
features_selected_2=importances_6.loc[importances_6['importance_gain']>importances_6['quantile_0.95'],'feature'].tolist()

print(len(features_selected_2))

importancias_7=entrena_lgb(data=df_data,features=features_selected_2,categorical=categorical,target='readmitted')

Vemos que el target encoding incluso llega a descartar variables que estan adelante en el ranking de importancias, como 'var_enconding_diag_1-readmitted'.Finalmente con el target permutation obtenemos un modelo de solo 30 variables con un score que supera a la baseline y de casi el mismo score que el mejor modelo en los experimentos realizados.

Las variables seleccionadas en orden de relevancia para el modelo bajo los test respectivos son las siguientes:

In [42]:
importancias_7['variable'].tolist()

Las variales que se descartaron fueron:


In [43]:
no_usar=['encounter_id','readmitted','estimator']

variables=[x for x in df_data.columns if x not in no_usar]

variables_descartadas=[x for x in variables if x not in features_selected_2]

variables_descartadas